# Spam Classification

In [56]:
import numpy as np

In [57]:
data = np.loadtxt('./spambase/spambase.data', delimiter=',')
print("Data layout: ", data.shape)
print("A single sample: ", data[0])

Data layout:  (4601, 58)
A single sample:  [  0.      0.64    0.64    0.      0.32    0.      0.      0.      0.
   0.      0.      0.64    0.      0.      0.      0.32    0.      1.29
   1.93    0.      0.96    0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.778   0.      0.
   3.756  61.    278.      1.   ]


We will split X and Y labels 

Here label 1 : Spam
           0 : Not Spam

In [58]:
X = data[:,0:-1]
Y = data[:,-1]

In [62]:
#Split Train data into 10- folds
from sklearn.model_selection import KFold
no_of_folds = 10
kf = KFold(n_splits=no_of_folds, shuffle = True) #Splits data into 10 folds and shuffle data before splitting

    

In [63]:
#Create Multinomial Naive Bayes model

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=1, fit_prior = True) #alpha : laplace smoothing parameter to handle unseen test data, fit_prior to learn class prior probabilities
i = 1
result = []
running_sum_accuracy = 0
running_sum_fp = 0
running_sum_fn = 0

for train_index, test_index in kf.split(X): #train_index and test_index contains list of indices pointing to data in X
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model.fit(X_train, y_train) #Learn the conditional probabilties P(word|class) from the train data
    predicted_labels = model.predict(X_test) #Predicts probability of each instance belonging to class Spam or Not Spam
   
    FP = np.sum(np.logical_and(predicted_labels == 1, y_test == 0)) #False Positiveis number of emails incorrectly classifies as Spam
    FPR = FP/np.sum(y_test==0) #False Positive Rate
    
    FN = np.sum(np.logical_and(predicted_labels == 0, y_test == 1)) #False Negative is number of emails incorrectly clsssified as Not Spam
    FNR = FN/np.sum(y_test==1) #False Negative rate
    
    OER = (FP + FN)/len(y_test) #Overall Error Rate(OER) : %age of total examples that were misclassified
    result.append([i,FPR,FNR,OER])
    
    #For Averaging results
    running_sum_accuracy += OER 
    running_sum_fp += FPR
    running_sum_fn += FNR  
    i+=1
result.append(["Average Rate", running_sum_fp/no_of_folds, running_sum_fn/no_of_folds, running_sum_accuracy/no_of_folds])


In [64]:
#Printing the results
import pandas as pd
df = pd.DataFrame(result, columns = ["Fold","False Positive", "False Negative", "Overall Error Rate"])
df.set_index('Fold', inplace=True)
df

,False Positive,False Negative,Overall Error Rate
Fold,,,
1,0.164234,0.283422,0.212581
2,0.151079,0.285714,0.204348
3,0.140741,0.326316,0.217391
4,0.205993,0.290155,0.241304
5,0.183099,0.261364,0.213043
6,0.135531,0.256684,0.184783
7,0.168421,0.245714,0.197826
8,0.147368,0.245714,0.184783
9,0.152249,0.257310,0.191304
